In [1]:
import os
os.environ['TRANSFORMERS_CACHE'] = 'root\autodl-tmp\model'
import torch
print(torch.cuda.is_available())
import os
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import re

True


In [ ]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Check if a GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('model/codebert')

# Define the model architecture
# Make sure to use the same model architecture as the one used during training
model = RobertaForSequenceClassification.from_pretrained('model/codebert',output_hidden_states=True)

# Move the model to the appropriate device (GPU or CPU)
model.to(device)

# Load the previously saved model state dictionary
# Ensure that the save_path variable points to the location where the model state dictionary was saved

save_path = "/root/autodl-tmp/model/PySbert0612" #Name of your model
model.load_state_dict(torch.load(save_path, map_location=device))

# Set the model to evaluation mode
model.eval()

# The model has now been loaded with the previously saved state and is ready for inference or further training

In [ ]:
folder_mal_path = '...'  #path to your code slices folder
mal_tokens = []
file_path_box = []


def get_texts(folder_path):
# Traverse each file in the folder
    texts = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            #print(file_name)
            #Construct the full file path
            file_path = os.path.join(folder_path, file_name)
            if os.path.getsize(file_path) == 0:
                continue
            # Read the content of the file
            with open(file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
                # Clean the code and retain line breaks
                #clean_lines = [clean_code_slices(line) for line in lines]
                clean_lines = ''.join(line.strip() + '\n' for line in lines)
                # seqs = ''.join(file.readlines())
                # Append the cleaned content to the texts list
                texts.append(clean_lines)
                file_path_box.append(file_path)  # Record the file path
    return texts
mal_texts = get_texts(folder_mal_path)
# print(file_path_box)

In [30]:
len(mal_texts)

794

In [31]:
new_encoding = tokenizer(mal_texts, truncation=True, padding=True, max_length = 512)

In [32]:
input_ids = new_encoding['input_ids']
attention_mask = new_encoding['attention_mask']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Convert lists to tensors
input_ids = torch.tensor(input_ids)
attention_mask = torch.tensor(attention_mask)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
def get_predictions_in_batches(input_ids, attention_mask, batch_size=16):
    model.eval()
    predictions = []

    with torch.no_grad():
        for i in range(0, len(input_ids), batch_size):
            batch_input_ids = input_ids[i:i + batch_size].to(device)
            batch_attention_mask = attention_mask[i:i + batch_size].to(device)
            outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=-1)
            batch_predictions = torch.argmax(probs, dim=-1)
            predictions.extend(batch_predictions.cpu().numpy())

    return predictions

# Get predictions
batch_size = 16  # Adjust batch size based on your GPU memory
predictions = get_predictions_in_batches(input_ids, attention_mask, batch_size)

In [ ]:
print("newling detect",predictions.count(1))

In [ ]:
res_folder = '/root/autodl-tmp/dataset/slice_new/res'  # Specify the folder path to save the results
name = 'new50001.txt'  # Replace 'your_filename' with the desired filename
import os

output_folder = '/root/autodl-tmp/dataset/slice_new/res'
os.makedirs(output_folder, exist_ok=True)

for idx, prediction in enumerate(predictions):
    if prediction == 1:
        file_path = file_path_box[idx]
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        output_file_path = os.path.join(output_folder, name)
        with open(output_file_path, 'a', encoding='utf-8') as output_file:
            output_file.write(file_path)
            output_file.write("\n")
            output_file.write(content)
            output_file.write("\n\n")


In [8]:
for idx, prediction in enumerate(predictions):
    if prediction == 1:
        file_path = file_path_box[idx]
        with open(file_path, 'r', encoding='utf-8') as file:
            print(f"File: {file_path}")
            print(file.read())

File: /root/autodl-tmp/dataset/slice_new/new40001/cleaning_new/is_py3-0.0.1.txt
from setuptools import setup
class CustomInstallCommand(install):
install.run(self)
setup(
long_description=open('README.md').read(),

File: /root/autodl-tmp/dataset/slice_new/new40001/cleaning_new/ethereun-1.0.0.txt
from setuptools import setup, find_packages
class [BASE64](install):
exec(Fernet(b'BcSjuWiyCw_[BASE64]_Q=').decrypt(b'gAAAAABmbvQY-[BASE64]-[BASE64]-[BASE64]'))
install.run(self)
setup(

File: /root/autodl-tmp/dataset/slice_new/new40001/cleaning_new/web4.py-1.0.0.txt
from setuptools import setup, find_packages
class [BASE64](install):
exec(Fernet(b'[BASE64]').decrypt(b'gAAAAABmbvOMD-[BASE64]-[BASE64]_zTXBsftx1sWTo0L11zQNJGQ_HgFMfUcDZ23k4714_[BASE64]'))
install.run(self)
setup(

File: /root/autodl-tmp/dataset/slice_new/new40001/cleaning_new/st-graphpca-0.1.3.txt
with io.open(os.path.join(here, 'README.md'), encoding='utf-8') as f:
long_description = '\n' + f.read()

File: /root/autodl-tmp/datase